# TAR: Proyecto Final 

## Tratamiento de datos

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
ruta_datosMOP = "./Datos/MOP"
ruta_datosADME = "./Datos/ADME"
ruta_datos_procesados = "./Datos_procesados"


## Entorno personalizado para el problema hidro-térmico

In [ ]:
%pip install gymnasium

In [ ]:
import gymnasium as gym
from gymnasium import spaces

V_MAX = 0
N_HIDRO = 0
V0 = 0

class HydroThermalEnv(gym.Env):
    def __init__(self):
        # Defino espacios de observación y acción
        self.observation_space = spaces.Dict({
            "volumen": spaces.Box(0.0, V_MAX, shape=()),
            "hidrologia": spaces.Discrete(N_HIDRO),
            "tiempo": spaces.Discrete(52)
        })
        # turbinar en [0, V_MAX] continuo o discretizado
        self.action_space = spaces.Box(0.0, V_MAX, shape=())
        ...
    def reset(self, *, seed=None, options=None):
        self.v = V0
        self.t = 0
        self.h = inicial_hidrologia()
        return self._get_obs(), {}
    def step(self, action):
        qt = float(action)
        # dinámica: v←v−q−d+a
        self.v = np.clip(self.v - qt - self._vertido(qt) + self._aportes(), 0, V_MAX)
        self.h = siguiente_hidrologia(self.t, self.h)
        self.t += 1
        # recompensa = −costo_termo(demanda_restante)
        reward = -costo_termico(self.demanda(), qt, self.v)
        done = (self.t >= T_MAX)
        return self._get_obs(), reward, done, False, {}
    def render(self): ...
    def _get_obs(self):
        return {"volumen": self.v, "hidrologia": self.h, "tiempo": self.t}